In [25]:
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

In [ ]:
sevMil = pd.read_csv("companies_sorted.csv")
ft = pd.read_csv("Forbes Top2000 2017.csv")
db = pd.read_csv("sparql_2022-10-02_13-24-07Z.csv")

In [10]:
sevMil = sevMil[['name']]
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1M_" + sevMil['id'].astype(str)

In [12]:
ft = ft[['Company']]
ft['id'] =  range(len(ft))
ft['id'] = "ft_" + ft['id'].astype(str)

In [ ]:
dbpedia = db[['name']]
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "db_" + dbpedia['id'].astype(str)

In [27]:
new_df = pd.DataFrame(product(ft['Company'], dbpedia['name']), columns=["ft","dbpedia"])

new_df["LevScore"] = new_df.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

In [ ]:
new_df2 = new_df[new_df['LevScore'] > 0.8]

In [ ]:
confirmed_true = new_df2[new_df2['LevScore'] == 1]
edge_cases = new_df2[new_df2['LevScore'] != 1]

In [ ]:
true_sample = confirmed_true.sample(200)
false_sample = edge_cases.sample(200)

In [73]:
gold_standard_ft_db = pd.concat([true_sample,false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='Company')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]


,ft_id,db_id,LevScore,ft,dbpedia
0,ft_33,db_5471,1.000000,Bank of Communications,Bank of Communications
1,ft_1578,db_3816,1.000000,Royal Mail,Royal Mail
2,ft_1819,db_9442,1.000000,Umicore,Umicore
3,ft_1204,db_4662,1.000000,SCANA,SCANA
4,ft_1215,db_3083,1.000000,Avnet,Avnet
...,...,...,...,...,...
402,ft_1199,db_3122,0.842105,Arab National Bank,Punjab National Bank
403,ft_409,db_3267,0.820513,Magna International,Harman International
404,ft_1310,db_9877,0.827586,Dxc Technology,Beck Technology
405,ft_52,db_3782,0.888889,ING Group,PIN Group
